# 自动并行

在延后执行里我们提到后端系统会自动构建计算图。通过计算图系统可以知道所有计算的依赖关系，有了它系统可以选择将没有依赖关系任务同时执行来获得性能的提升。

仍然考虑下面这个例子，这里``a = ...``和``b = ...``之间没有数据依赖关系，从而系统可以选择并行执行他们。

<img src="http://zh.gluon.ai/_images/frontend-backend.svg" width="400">

通常一个运算符，例如+或者dot，会用掉一个计算设备上所有计算资源。dot同样用掉所有CPU的核（即使是有多个CPU）和单GPU上所有线程。因此在单设备上并行运行多个运算符可能效果并不明显。自动并行主要的用途是多设备的计算并行，和计算与通讯的并行。

## 多设备的并行计算

我们首先定义一个函数，它做10次矩阵乘法。

In [1]:
from mxnet import nd

def run(X):
    return [nd.dot(X, X) for i in range(10)]

我们分别计算在CPU和GPU上运行时间

In [5]:
import mxnet as mx
from time import time 

x_cpu = nd.random.normal(shape=(2000, 2000))
x_gpu = nd.random.normal(shape=(6000, 6000), ctx=mx.gpu())

# warm up
run(x_cpu)
run(x_gpu)
nd.waitall()

start = time()
run(x_cpu)
nd.waitall()
print("Run on cpu %f sec" % (time() - start))

start = time()
run(x_gpu)
nd.waitall()
print("Run on gpu %f sec" % (time() - start))

Run on cpu 0.870323 sec
Run on gpu 0.413594 sec


我们去掉两次``run``之间的``waitall``，希望系统能自动并行这两个任务：

In [6]:
start = time()
run(x_cpu)
run(x_gpu)
nd.waitall()
print("Run on both CPU and GPU %f sec" % (time() - start))

Run on both CPU and GPU 0.895741 sec


可以看到当两个一起执行时，总时间不是分开执行的总和，这表示后端系统能够有效的并行他们。

## 计算和通讯的并行

在多设备计算中，我们经常需要在设备之间复制数据，例如我们呢在GPU上计算，然后将结果复制会CPU。

In [10]:
def copy_to_cpu(x):
    return [y.copyto(mx.cpu()) for y in x]

start = time()
y = run(x_gpu)
nd.waitall()
print("Run on gpu %f sec" % (time() - start))

start = time()
copy_to_cpu(y)
nd.waitall()
print("Copy to cpu %f sec" % (time() - start))

Run on gpu 0.432925 sec
Copy to cpu 0.471936 sec


同样我们去掉nd.waitall，来看看并行的效果

In [11]:
start = time()
y = run(x_gpu)
copy_to_cpu(y)
nd.waitall()
print('Run on GPU then Copy to CPU: %f sec'%(time() - start))

Run on GPU then Copy to CPU: 0.522506 sec


可以看到总时间小于前面两者之和。这个任务稍微不同于上面，因为运行和复制之间有依赖关系。就是y[i]必须先计算好才能复制到CPU。**但在计算y[i]的时候系统可以复制y[i-1]，从而获得总运行时间的减少。**

## 总结

MXNet能够自动并行执行没有数据依赖关系的任务从而提升系统性能。